## Autogen Discord QA

I want to develop a LLM Application that will let me ask any question with regards to Autogen. The application should be able to:
1. Search discord history for answers
2. Multi model capabilities to retrieve images also.
3. Search the internet to get additional information if needed.

To pull message history, I am using an extension called [Discrub](https://chrome.google.com/webstore/detail/discrub/plhdclenpaecffbcefjmpkkbdpkmhhbj/related) as i had issues setting up the Discord API. <br>
Data pull is up until `15th November 2023`.

In [ ]:
import os
import sys
import json
import tempfile
from pathlib import Path
sys.path.append('./')

import openai
import autogen
import chromadb

config_list = autogen.config_list_from_dotenv(dotenv_file_path='../../.env')

In [ ]:
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Create an instance of RetrieveAssistantAgent
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
)

path = Path(os.getcwd(), 'docs')
client = chromadb.PersistentClient(path=f"{os.getcwd()}/chromadb")

# Create an instance of RetrieveUserProxyAgent
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "task": "default",
        "docs_path": str(path), 
        "chunk_token_size": 2000,
        "model": config_list[0]["model"],
        "client": client, 
        "embedding_model": "all-mpnet-base-v2",
    },
)


In [ ]:
# NOTE: Delete dir each instantiation
client.delete_collection('autogen-docs')

In [ ]:
# From a txt document 
user_question = "How can chainlit be used with Autogen? Provide a code example where appropiate"
ragproxyagent.initiate_chat(assistant, problem=user_question, clear_history=True) 
# ragproxyagent.send(recipient=assistant, message=user_question)

In [ ]:
# Print documents retrieved from Query
ragproxyagent.retrieve_docs("Chainlit")
ragproxyagent._results['documents']